In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline 


from fraud import collectAllFeaturesBaseline
df_fraud_aggregated = collectAllFeaturesBaseline() 

In [5]:
X = df_fraud_aggregated.drop(columns=["target", "client_id"])
X.fillna(0, inplace=True)  
y = df_fraud_aggregated["target"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=42
)


scale_pos_weight = (y_train == 0).sum() / (y_train == 1).sum()

pipeline_smote = Pipeline([
    ("smote", SMOTE(random_state=42, sampling_strategy="auto")),
    ("xgb", XGBClassifier(
        use_label_encoder=False,   # neuere XGBoost-Versionen
        eval_metric="logloss",     # oder "auc"
        n_estimators=150,          # Anzahl der Bäume
        max_depth=6,               # maximale Tiefe der Bäume
        learning_rate=0.1,         # Shrinkage
        subsample=0.8,             # Bagging-Rate
        colsample_bytree=0.8,      # Feature-Subsampling
        scale_pos_weight=scale_pos_weight,
        random_state=42
    ))
])

pipeline_smote.fit(X_train, y_train)
y_pred_smote = pipeline_smote.predict(X_test)

print(confusion_matrix(y_test, y_pred_smote))
print("=== SMOTE + XGBClassifier ===")
print(classification_report(y_test, y_pred_smote))




/Users/muri/education/spiced/WaterMLOps/ds-mlproject-fraud/.venv/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [22:44:42] WARNING: /private/var/folders/49/1y5t6tjd2hxgddgr4h8l86lr0000gn/T/pip-install-xwm7t153/xgboost_f76b7daff02d438b84bd67ed3f83a334/cpp_src/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


[[14939 23439]
 [  339  1931]]
=== SMOTE + XGBClassifier ===
              precision    recall  f1-score   support

         0.0       0.98      0.39      0.56     38378
         1.0       0.08      0.85      0.14      2270

    accuracy                           0.42     40648
   macro avg       0.53      0.62      0.35     40648
weighted avg       0.93      0.42      0.53     40648

